In [34]:
import pandas as pd
import numpy as np
from pathlib import Path

In [35]:
# Define the paths to the data files
data_folder = Path("../data/raw")
amikas_file = data_folder / "amikas_processed_data.csv"
predictions_file = data_folder / "predictions_2010_2025_for_arosha.csv"

print(f"Amikas data file path: {amikas_file}")
print(f"Predictions file path: {predictions_file}")

Amikas data file path: ..\data\raw\amikas_processed_data.csv
Predictions file path: ..\data\raw\predictions_2010_2025_for_arosha.csv


In [36]:
# Load the amikas processed data
try:
    amikas_df = pd.read_csv(amikas_file)
    print("Amikas data loaded successfully!")
    print(f"Shape of amikas data: {amikas_df.shape}")
    print(f"Columns in amikas data: {amikas_df.columns.tolist()}")
except FileNotFoundError:
    print(f"Error: Could not find file at {amikas_file}")

Amikas data loaded successfully!
Shape of amikas data: (66143, 19)
Columns in amikas data: ['date', 'tmax', 'tmin', 'precipitation', 'rh_mean', 'wind_speed', 'solar_radiation', 'spi_3', 'spi_6', 'heat_index', 'consecutive_dry_days', 'gdd', 'monsoon_season', 'city', 'latitude', 'longitude', 'climate_zone', 'elevation', 'monsoon_type']


In [37]:
# Load the predictions data
try:
    predictions_df = pd.read_csv(predictions_file)
    print("Predictions data loaded successfully!")
    print(f"Shape of predictions data: {predictions_df.shape}")
    print(f"Columns in predictions data: {predictions_df.columns.tolist()}")
except FileNotFoundError:
    print(f"Error: Could not find file at {predictions_file}")

Predictions data loaded successfully!
Shape of predictions data: (40572, 6)
Columns in predictions data: ['date', 'city', 'prob_drought', 'prob_flood_risk', 'prob_normal', 'actual_class']


In [38]:
# Check the data types and first few rows of both datasets
print("Amikas data info:")
print(amikas_df.info())
print("\nFirst 5 rows of amikas data:")
print(amikas_df.head())

Amikas data info:
<class 'pandas.DataFrame'>
RangeIndex: 66143 entries, 0 to 66142
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  66143 non-null  str    
 1   tmax                  66143 non-null  float64
 2   tmin                  66143 non-null  float64
 3   precipitation         66143 non-null  float64
 4   rh_mean               66143 non-null  float64
 5   wind_speed            66143 non-null  float64
 6   solar_radiation       66143 non-null  float64
 7   spi_3                 65723 non-null  float64
 8   spi_6                 65079 non-null  float64
 9   heat_index            66143 non-null  float64
 10  consecutive_dry_days  66143 non-null  int64  
 11  gdd                   66143 non-null  float64
 12  monsoon_season        66143 non-null  str    
 13  city                  66143 non-null  str    
 14  latitude              66143 non-null  float64
 15  longitude   

In [39]:
print("Predictions data info:")
print(predictions_df.info())
print("\nFirst 5 rows of predictions data:")
print(predictions_df.head())

Predictions data info:
<class 'pandas.DataFrame'>
RangeIndex: 40572 entries, 0 to 40571
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             40572 non-null  str    
 1   city             40572 non-null  str    
 2   prob_drought     40572 non-null  float64
 3   prob_flood_risk  40572 non-null  float64
 4   prob_normal      40572 non-null  float64
 5   actual_class     40572 non-null  str    
dtypes: float64(3), str(3)
memory usage: 1.9 MB
None

First 5 rows of predictions data:
       date          city  prob_drought  prob_flood_risk  prob_normal  \
0  1/1/2010  Nuwara_Eliya          0.90              0.0         0.10   
1  1/2/2010  Nuwara_Eliya          0.96              0.0         0.04   
2  1/3/2010  Nuwara_Eliya          0.96              0.0         0.04   
3  1/4/2010  Nuwara_Eliya          0.99              0.0         0.01   
4  1/5/2010  Nuwara_Eliya          0.98              0.0   

In [40]:
# Convert date columns to datetime format for proper matching
amikas_df['date'] = pd.to_datetime(amikas_df['date'])
predictions_df['date'] = pd.to_datetime(predictions_df['date'])

print("Date columns converted to datetime format")
print(f"Amikas date range: {amikas_df['date'].min()} to {amikas_df['date'].max()}")
print(f"Predictions date range: {predictions_df['date'].min()} to {predictions_df['date'].max()}")

Date columns converted to datetime format
Amikas date range: 2000-01-01 00:00:00 to 2025-11-13 00:00:00
Predictions date range: 2010-01-01 00:00:00 to 2025-11-13 00:00:00


In [41]:
# Create a dictionary with (date, city) as keys and precipitation as values
# This approach is memory efficient for matching

# Convert date to string for dictionary keys to ensure consistency
date_city_to_precip = {}

for idx, row in amikas_df.iterrows():
    key = (row['date'].strftime('%Y-%m-%d'), row['city'])
    date_city_to_precip[key] = row['precipitation']

print(f"Created mapping dictionary with {len(date_city_to_precip)} unique (date, city) combinations")

Created mapping dictionary with 66143 unique (date, city) combinations


In [42]:
# Initialize the precipitation column with NaN (null values)
predictions_df['precipitation'] = np.nan

# Function to get precipitation value for each row
def get_precipitation(row):
    key = (row['date'].strftime('%Y-%m-%d'), row['city'])
    return date_city_to_precip.get(key, np.nan)

# Apply the function to create the precipitation column
predictions_df['precipitation'] = predictions_df.apply(get_precipitation, axis=1)

print("Precipitation column created and populated")

Precipitation column created and populated


In [43]:
# Check how many matches were found
matches_found = predictions_df['precipitation'].notna().sum()
total_rows = len(predictions_df)
match_percentage = (matches_found / total_rows) * 100

print(f"Total rows in predictions dataset: {total_rows}")
print(f"Rows with precipitation data matched: {matches_found}")
print(f"Match percentage: {match_percentage:.2f}%")

Total rows in predictions dataset: 40572
Rows with precipitation data matched: 40572
Match percentage: 100.00%


In [44]:
# Display sample rows with matches
print("Sample rows where precipitation was found:")
matched_samples = predictions_df[predictions_df['precipitation'].notna()].head(10)
print(matched_samples[['date', 'city', 'precipitation']])

Sample rows where precipitation was found:
        date          city  precipitation
0 2010-01-01  Nuwara_Eliya           0.05
1 2010-01-02  Nuwara_Eliya           0.47
2 2010-01-03  Nuwara_Eliya           8.10
3 2010-01-04  Nuwara_Eliya           0.21
4 2010-01-05  Nuwara_Eliya           0.00
5 2010-01-06  Nuwara_Eliya           0.00
6 2010-01-07  Nuwara_Eliya           0.33
7 2010-01-08  Nuwara_Eliya           9.10
8 2010-01-09  Nuwara_Eliya          11.74
9 2010-01-10  Nuwara_Eliya           1.54


In [45]:
# Display sample rows without matches (if any)
if matches_found < total_rows:
    print("\nSample rows where precipitation was NOT found:")
    unmatched_samples = predictions_df[predictions_df['precipitation'].isna()].head(10)
    print(unmatched_samples[['date', 'city']])

In [46]:
# Pick a few random rows from predictions and verify they match with amikas data
sample_indices = np.random.choice(len(predictions_df), min(5, len(predictions_df)), replace=False)

for idx in sample_indices:
    pred_row = predictions_df.iloc[idx]
    if pd.notna(pred_row['precipitation']):
        # Find matching row in amikas data
        match = amikas_df[(amikas_df['date'] == pred_row['date']) &
                          (amikas_df['city'] == pred_row['city'])]

        if not match.empty:
            print(f"\nDate: {pred_row['date'].strftime('%Y-%m-%d')}, City: {pred_row['city']}")
            print(f"  Precipitation in predictions: {pred_row['precipitation']}")
            print(f"  Precipitation in amikas: {match.iloc[0]['precipitation']}")


Date: 2015-12-16, City: Badulla
  Precipitation in predictions: 1.11
  Precipitation in amikas: 1.11

Date: 2016-03-20, City: Ratnapura
  Precipitation in predictions: 0.03
  Precipitation in amikas: 0.03

Date: 2010-05-08, City: Kurunegala
  Precipitation in predictions: 10.98
  Precipitation in amikas: 10.98

Date: 2021-11-28, City: Nuwara_Eliya
  Precipitation in predictions: 10.98
  Precipitation in amikas: 10.98

Date: 2014-10-20, City: Hambantota
  Precipitation in predictions: 31.97
  Precipitation in amikas: 31.97


In [47]:
# Save the updated predictions dataframe back to CSV
output_file = data_folder / "predictions_2010_2025_for_arosha_updated.csv"
predictions_df.to_csv(output_file, index=False)
print(f"Updated predictions file saved to: {output_file}")

Updated predictions file saved to: ..\data\raw\predictions_2010_2025_for_arosha_updated.csv


In [48]:
# Load the CSV file
input_path = "../data/raw/predictions_2010_2025_for_arosha_updated.csv"
df = pd.read_csv(input_path)

# Display basic information
print("Dataset shape:", df.shape)
df.head()

Dataset shape: (40572, 7)


,date,city,prob_drought,prob_flood_risk,prob_normal,actual_class,precipitation
0,2010-01-01,Nuwara_Eliya,0.90,0.0,0.10,drought,0.05
1,2010-01-02,Nuwara_Eliya,0.96,0.0,0.04,drought,0.47
2,2010-01-03,Nuwara_Eliya,0.96,0.0,0.04,drought,8.10
3,2010-01-04,Nuwara_Eliya,0.99,0.0,0.01,drought,0.21
4,2010-01-05,Nuwara_Eliya,0.98,0.0,0.02,drought,0.00


In [49]:
# Check data types, missing values, and unique cities
df.info()
print("\nUnique cities:", df['city'].unique())
print("\nDate range:", df['date'].min(), "to", df['date'].max())

<class 'pandas.DataFrame'>
RangeIndex: 40572 entries, 0 to 40571
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             40572 non-null  str    
 1   city             40572 non-null  str    
 2   prob_drought     40572 non-null  float64
 3   prob_flood_risk  40572 non-null  float64
 4   prob_normal      40572 non-null  float64
 5   actual_class     40572 non-null  str    
 6   precipitation    40572 non-null  float64
dtypes: float64(4), str(3)
memory usage: 2.2 MB

Unique cities: <StringArray>
['Nuwara_Eliya',      'Badulla',       'Matale',    'Ratnapura',
       'Jaffna',   'Hambantota',   'Kurunegala']
Length: 7, dtype: str

Date range: 2010-01-01 to 2025-11-13


In [50]:
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 40572 entries, 0 to 40571
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             40572 non-null  datetime64[us]
 1   city             40572 non-null  str           
 2   prob_drought     40572 non-null  float64       
 3   prob_flood_risk  40572 non-null  float64       
 4   prob_normal      40572 non-null  float64       
 5   actual_class     40572 non-null  str           
 6   precipitation    40572 non-null  float64       
dtypes: datetime64[us](1), float64(4), str(2)
memory usage: 2.2 MB


In [51]:
from datetime import datetime, timedelta


def get_week_code(dt):
    """
    Assign week code (W1..W52) based on fixed weekly intervals for the given year.
    Week 1: Jan 1 - Jan 7
    Week 2: Jan 8 - Jan 14
    ...
    Week 52: Dec 24 - Dec 30
    """
    year = dt.year
    # Iterate over possible weeks 1 to 52
    for week_num in range(1, 53):
        start = datetime(year, 1, 1) + timedelta(days=(week_num - 1) * 7)
        end = start + timedelta(days=6)
        if start <= dt <= end:
            return f"W{week_num}"
    return None  # Date not in any week (e.g., Dec 31)

In [52]:
df['week_code'] = df['date'].apply(get_week_code)

# Check how many rows were assigned a week code
print("Rows with week code assigned:", df['week_code'].notna().sum())
print("Rows without week code:", df['week_code'].isna().sum())

# Show a few examples
df[['date', 'week_code']].head(10)

Rows with week code assigned: 40439
Rows without week code: 133


,date,week_code
0,2010-01-01,W1
1,2010-01-02,W1
2,2010-01-03,W1
3,2010-01-04,W1
4,2010-01-05,W1
5,2010-01-06,W1
6,2010-01-07,W1
7,2010-01-08,W2
8,2010-01-09,W2
9,2010-01-10,W2


In [53]:
# Keep only rows that belong to a defined week
df_weekly = df.dropna(subset=['week_code']).copy()
print("New shape after filtering:", df_weekly.shape)

New shape after filtering: (40439, 8)


In [54]:
# Example: Check dates for year 2020 (leap year) to ensure weeks align with specification
year_check = 2020
check_dates = df_weekly[df_weekly['date'].dt.year == year_check][['date', 'week_code']].drop_duplicates().sort_values('date')
print(f"Week boundaries for {year_check}:")
print(check_dates.head(14))  # First two weeks
print("...")
print(check_dates.tail(14))  # Last two weeks

Week boundaries for 2020:
           date week_code
3652 2020-01-01        W1
3653 2020-01-02        W1
3654 2020-01-03        W1
3655 2020-01-04        W1
3656 2020-01-05        W1
3657 2020-01-06        W1
3658 2020-01-07        W1
3659 2020-01-08        W2
3660 2020-01-09        W2
3661 2020-01-10        W2
3662 2020-01-11        W2
3663 2020-01-12        W2
3664 2020-01-13        W2
3665 2020-01-14        W2
...
           date week_code
4002 2020-12-16       W51
4003 2020-12-17       W51
4004 2020-12-18       W51
4005 2020-12-19       W51
4006 2020-12-20       W51
4007 2020-12-21       W51
4008 2020-12-22       W51
4009 2020-12-23       W52
4010 2020-12-24       W52
4011 2020-12-25       W52
4012 2020-12-26       W52
4013 2020-12-27       W52
4014 2020-12-28       W52
4015 2020-12-29       W52


In [55]:
# Extract year from date
df_weekly['year'] = df_weekly['date'].dt.year

# Define custom aggregation for actual_class
def weekly_actual_class(series):
    if 'flood_risk' in series.values:
        return 'flood_risk'
    else:
        # Get the mode(s) and return the first one
        modes = series.mode()
        if len(modes) > 0:
            return modes.iloc[0]
        else:
            return None  # Should not happen if group is non-empty

# Group by city, year, and week_code
grouped = df_weekly.groupby(['city', 'year', 'week_code']).agg(
    prob_drought=('prob_drought', 'mean'),
    prob_flood_risk=('prob_flood_risk', 'mean'),
    prob_normal=('prob_normal', 'mean'),
    precipitation=('precipitation', 'mean'),
    actual_class=('actual_class', weekly_actual_class)
).reset_index()

# Display a sample of the grouped data
grouped.head()

,city,year,week_code,prob_drought,prob_flood_risk,prob_normal,precipitation,actual_class
0,Badulla,2010,W1,0.995714,0.000000,0.004286,2.890000,drought
1,Badulla,2010,W10,0.000000,0.000000,1.000000,0.048571,normal
2,Badulla,2010,W11,0.000000,0.000000,1.000000,0.131429,normal
3,Badulla,2010,W12,0.001429,0.007143,0.991429,2.041429,normal
4,Badulla,2010,W13,0.000000,0.138571,0.861429,12.085714,flood_risk


In [56]:
print("Final weekly dataset shape:", grouped.shape)
print("\nNumber of records per city:")
print(grouped['city'].value_counts())
print("\nSummary statistics for numeric columns:")
grouped.describe()

Final weekly dataset shape: (5782, 8)

Number of records per city:
city
Badulla         826
Hambantota      826
Jaffna          826
Kurunegala      826
Matale          826
Nuwara_Eliya    826
Ratnapura       826
Name: count, dtype: int64

Summary statistics for numeric columns:


,year,prob_drought,prob_flood_risk,prob_normal,precipitation
count,5782.000000,5782.000000,5782.000000,5782.000000,5782.000000
mean,2017.445521,0.239433,0.046769,0.713798,5.471958
std,4.582186,0.388843,0.104075,0.378335,5.969009
min,2010.000000,0.000000,0.000000,0.000000,0.000000
25%,2013.000000,0.001429,0.000000,0.478571,1.196071
50%,2017.000000,0.008571,0.000000,0.947143,3.650000
75%,2021.000000,0.424286,0.012500,0.995714,7.743571
max,2025.000000,1.000000,0.801429,1.000000,50.140000


In [57]:
output_path = "../data/raw/amikas_prepared_output.csv"
grouped.to_csv(output_path, index=False)
print(f"File saved successfully to: {output_path}")

File saved successfully to: ../data/raw/amikas_prepared_output.csv


In [58]:
# Load climate/weather data
climate_df = pd.read_csv('../data/raw/amikas_prepared_output.csv')

# Load vegetable price data
price_df = pd.read_excel('../data/raw/Amika.xlsx')

# Display first few rows to understand structure
print("Climate data sample:")
display(climate_df.head())

print("\nPrice data sample:")
display(price_df.head())

Climate data sample:


,city,year,week_code,prob_drought,prob_flood_risk,prob_normal,precipitation,actual_class
0,Badulla,2010,W1,0.995714,0.000000,0.004286,2.890000,drought
1,Badulla,2010,W10,0.000000,0.000000,1.000000,0.048571,normal
2,Badulla,2010,W11,0.000000,0.000000,1.000000,0.131429,normal
3,Badulla,2010,W12,0.001429,0.007143,0.991429,2.041429,normal
4,Badulla,2010,W13,0.000000,0.138571,0.861429,12.085714,flood_risk



Price data sample:


,Year,Location,Item,W1,W2,W3,W4,W5,W6,W7,...,W44,W45,W46,W47,W48,W49,W50,W51,W52,W53
0,2010,Dambulla,Carrot,66.0,68.6,105.0,105.0,NaN,65.8,46.0,...,55.2,71.0,56.4,67.6,56.0,63.0,93.6,100.0,75.0,NaN
1,2010,Dambulla,Cabbage,38.2,41.0,47.5,47.5,NaN,35.0,29.8,...,32.2,30.0,33.0,26.8,22.6,25.6,35.0,44.6,53.0,NaN
2,2010,Dambulla,Tomatoes,51.0,71.6,58.0,58.0,NaN,78.6,56.0,...,53.0,49.6,66.0,54.0,45.4,46.0,56.0,49.0,52.4,NaN
3,2010,Dambulla,Brinjals,60.6,51.6,67.0,67.0,NaN,8.4,14.6,...,39.0,11.2,19.6,24.2,35.8,51.0,65.6,70.6,62.6,NaN
4,2010,Dambulla,Pumpkin,20.4,25.4,22.5,22.5,NaN,15.0,12.6,...,33.8,19.2,24.4,19.6,20.2,22.2,19.2,12.2,27.4,NaN


In [59]:
# Check climate data info
print("Climate data info:")
climate_df.info()

print("\nUnique cities in climate data:")
print(climate_df['city'].unique())

print("\nUnique weeks in climate data:")
print(climate_df['week_code'].unique())

# Check price data info
print("\nPrice data info:")
price_df.info()

print("\nUnique items in price data:")
print(price_df['Item'].unique())

print("\nUnique years in price data:")
print(price_df['Year'].unique())

Climate data info:
<class 'pandas.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             5782 non-null   str    
 1   year             5782 non-null   int64  
 2   week_code        5782 non-null   str    
 3   prob_drought     5782 non-null   float64
 4   prob_flood_risk  5782 non-null   float64
 5   prob_normal      5782 non-null   float64
 6   precipitation    5782 non-null   float64
 7   actual_class     5782 non-null   str    
dtypes: float64(4), int64(1), str(3)
memory usage: 361.5 KB

Unique cities in climate data:
<StringArray>
[     'Badulla',   'Hambantota',       'Jaffna',   'Kurunegala',
       'Matale', 'Nuwara_Eliya',    'Ratnapura']
Length: 7, dtype: str

Unique weeks in climate data:
<StringArray>
[ 'W1', 'W10', 'W11', 'W12', 'W13', 'W14', 'W15', 'W16', 'W17', 'W18', 'W19',
  'W2', 'W20', 'W21', 'W22', 'W23', 'W24', 'W25', 'W26', 'W27

In [60]:
# Identify week columns (all columns that start with 'W' and then a number)
week_cols = [col for col in price_df.columns if col.startswith('W') and col[1:].isdigit()]

# Melt the dataframe
price_long = price_df.melt(
    id_vars=['Year', 'Item'],
    value_vars=week_cols,
    var_name='week',
    value_name='price'
)

# Convert week to string (it already is, but ensure it's consistent)
price_long['week'] = price_long['week'].astype(str)

# Sort for clarity
price_long = price_long.sort_values(['Year', 'week', 'Item']).reset_index(drop=True)

print("Reshaped price data (first 10 rows):")
display(price_long.head(10))

Reshaped price data (first 10 rows):


,Year,Item,week,price
0,2010,Bitter Gourd,W1,52.4
1,2010,Brinjals,W1,60.6
2,2010,Cabbage,W1,38.2
3,2010,Carrot,W1,66.0
4,2010,Pumpkin,W1,20.4
5,2010,Tomatoes,W1,51.0
6,2010,Bitter Gourd,W10,49.0
7,2010,Brinjals,W10,27.8
8,2010,Cabbage,W10,33.4
9,2010,Carrot,W10,43.6


In [61]:
# Pivot the climate data: index = year and week_code, columns = city, values = the variables
# This creates a MultiIndex in columns: (variable, city)
pivot_climate = climate_df.pivot_table(
    index=['year', 'week_code'],
    columns='city',
    values=['prob_drought', 'prob_flood_risk', 'prob_normal', 'precipitation', 'actual_class'],
    aggfunc='first'  # Assuming no duplicates; if duplicates exist, we need to decide how to handle
)

# Flatten the MultiIndex columns: combine variable and city
pivot_climate.columns = [f'{city}_{var}' for var, city in pivot_climate.columns]

# Reset index to make year and week_code regular columns
pivot_climate = pivot_climate.reset_index()

# Rename week_code to week to match price data
pivot_climate.rename(columns={'week_code': 'week'}, inplace=True)

print("Pivoted climate data (first 5 rows):")
display(pivot_climate.head())

Pivoted climate data (first 5 rows):


,year,week,Badulla_actual_class,Hambantota_actual_class,Jaffna_actual_class,Kurunegala_actual_class,Matale_actual_class,Nuwara_Eliya_actual_class,Ratnapura_actual_class,Badulla_precipitation,...,Matale_prob_flood_risk,Nuwara_Eliya_prob_flood_risk,Ratnapura_prob_flood_risk,Badulla_prob_normal,Hambantota_prob_normal,Jaffna_prob_normal,Kurunegala_prob_normal,Matale_prob_normal,Nuwara_Eliya_prob_normal,Ratnapura_prob_normal
0,2010,W1,drought,normal,drought,drought,drought,drought,normal,2.890000,...,0.000000,0.000000,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
1,2010,W10,normal,normal,normal,normal,normal,normal,normal,0.048571,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.998571,1.000000,1.000000,0.998571,0.990000
2,2010,W11,normal,normal,normal,normal,normal,normal,normal,0.131429,...,0.000000,0.000000,0.000000,1.000000,0.995714,0.994286,0.998571,0.998571,1.000000,0.994286
3,2010,W12,normal,normal,normal,normal,normal,normal,normal,2.041429,...,0.007143,0.031429,0.004286,0.991429,0.995714,0.994286,0.992857,0.991429,0.965714,0.988571
4,2010,W13,flood_risk,flood_risk,normal,flood_risk,flood_risk,flood_risk,flood_risk,12.085714,...,0.121429,0.250000,0.194286,0.861429,0.782857,0.990000,0.877143,0.877143,0.750000,0.804286


In [62]:
# Ensure year and week are same type (int for year, str for week)
price_long['year'] = price_long['Year'].astype(int)
pivot_climate['year'] = pivot_climate['year'].astype(int)

# Merge
combined_df = pd.merge(
    price_long,
    pivot_climate,
    left_on=['year', 'week'],
    right_on=['year', 'week'],
    how='left'  # left join to keep all price rows even if climate data missing for some week
)

# Drop the redundant 'Year' column if desired, keep 'year' for consistency
combined_df.drop(columns=['Year'], inplace=True)

# Reorder columns: year, week, vegetable, price, then all city columns
cols = ['year', 'week', 'Item', 'price'] + [col for col in combined_df.columns if col not in ['year', 'week', 'Item', 'price']]
combined_df = combined_df[cols]

# Rename 'Item' to 'vegetable' as requested
combined_df.rename(columns={'Item': 'vegetable'}, inplace=True)

print("Combined dataset (first 5 rows):")
display(combined_df.head())

print("\nColumn names:")
print(combined_df.columns.tolist())

Combined dataset (first 5 rows):


,year,week,vegetable,price,Badulla_actual_class,Hambantota_actual_class,Jaffna_actual_class,Kurunegala_actual_class,Matale_actual_class,Nuwara_Eliya_actual_class,...,Matale_prob_flood_risk,Nuwara_Eliya_prob_flood_risk,Ratnapura_prob_flood_risk,Badulla_prob_normal,Hambantota_prob_normal,Jaffna_prob_normal,Kurunegala_prob_normal,Matale_prob_normal,Nuwara_Eliya_prob_normal,Ratnapura_prob_normal
0,2010,W1,Bitter Gourd,52.4,drought,normal,drought,drought,drought,drought,...,0.0,0.0,0.0,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
1,2010,W1,Brinjals,60.6,drought,normal,drought,drought,drought,drought,...,0.0,0.0,0.0,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
2,2010,W1,Cabbage,38.2,drought,normal,drought,drought,drought,drought,...,0.0,0.0,0.0,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
3,2010,W1,Carrot,66.0,drought,normal,drought,drought,drought,drought,...,0.0,0.0,0.0,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
4,2010,W1,Pumpkin,20.4,drought,normal,drought,drought,drought,drought,...,0.0,0.0,0.0,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143



Column names:
['year', 'week', 'vegetable', 'price', 'Badulla_actual_class', 'Hambantota_actual_class', 'Jaffna_actual_class', 'Kurunegala_actual_class', 'Matale_actual_class', 'Nuwara_Eliya_actual_class', 'Ratnapura_actual_class', 'Badulla_precipitation', 'Hambantota_precipitation', 'Jaffna_precipitation', 'Kurunegala_precipitation', 'Matale_precipitation', 'Nuwara_Eliya_precipitation', 'Ratnapura_precipitation', 'Badulla_prob_drought', 'Hambantota_prob_drought', 'Jaffna_prob_drought', 'Kurunegala_prob_drought', 'Matale_prob_drought', 'Nuwara_Eliya_prob_drought', 'Ratnapura_prob_drought', 'Badulla_prob_flood_risk', 'Hambantota_prob_flood_risk', 'Jaffna_prob_flood_risk', 'Kurunegala_prob_flood_risk', 'Matale_prob_flood_risk', 'Nuwara_Eliya_prob_flood_risk', 'Ratnapura_prob_flood_risk', 'Badulla_prob_normal', 'Hambantota_prob_normal', 'Jaffna_prob_normal', 'Kurunegala_prob_normal', 'Matale_prob_normal', 'Nuwara_Eliya_prob_normal', 'Ratnapura_prob_normal']


In [66]:
# Step 7: Sort the Dataset by Year, Week (numerically), and Vegetable
# To sort weeks correctly (W1, W2, ..., W53), we extract the week number.
# Use raw string r'(\d+)' to avoid the SyntaxWarning.
combined_df['week_num'] = combined_df['week'].str.extract(r'(\d+)').astype(int)

# Sort by year, week_num, vegetable
combined_df = combined_df.sort_values(['year', 'week_num', 'vegetable']).reset_index(drop=True)

# Drop the helper column week_num
combined_df.drop(columns=['week_num'], inplace=True)

print("Sorted dataset (first 20 rows):")
display(combined_df.head(20))

Sorted dataset (first 20 rows):


,year,week,vegetable,price,Badulla_actual_class,Hambantota_actual_class,Jaffna_actual_class,Kurunegala_actual_class,Matale_actual_class,Nuwara_Eliya_actual_class,...,Matale_prob_flood_risk,Nuwara_Eliya_prob_flood_risk,Ratnapura_prob_flood_risk,Badulla_prob_normal,Hambantota_prob_normal,Jaffna_prob_normal,Kurunegala_prob_normal,Matale_prob_normal,Nuwara_Eliya_prob_normal,Ratnapura_prob_normal
0,2010,W1,Bitter Gourd,52.4,drought,normal,drought,drought,drought,drought,...,0.000000,0.0,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
1,2010,W1,Brinjals,60.6,drought,normal,drought,drought,drought,drought,...,0.000000,0.0,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
2,2010,W1,Cabbage,38.2,drought,normal,drought,drought,drought,drought,...,0.000000,0.0,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
3,2010,W1,Carrot,66.0,drought,normal,drought,drought,drought,drought,...,0.000000,0.0,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
4,2010,W1,Pumpkin,20.4,drought,normal,drought,drought,drought,drought,...,0.000000,0.0,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
5,2010,W1,Tomatoes,51.0,drought,normal,drought,drought,drought,drought,...,0.000000,0.0,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
6,2010,W2,Bitter Gourd,58.6,drought,normal,drought,drought,drought,drought,...,0.002857,0.0,0.001429,0.012857,0.911429,0.040000,0.047143,0.050000,0.055714,0.905714
7,2010,W2,Brinjals,51.6,drought,normal,drought,drought,drought,drought,...,0.002857,0.0,0.001429,0.012857,0.911429,0.040000,0.047143,0.050000,0.055714,0.905714
8,2010,W2,Cabbage,41.0,drought,normal,drought,drought,drought,drought,...,0.002857,0.0,0.001429,0.012857,0.911429,0.040000,0.047143,0.050000,0.055714,0.905714
9,2010,W2,Carrot,68.6,drought,normal,drought,drought,drought,drought,...,0.002857,0.0,0.001429,0.012857,0.911429,0.040000,0.047143,0.050000,0.055714,0.905714


In [67]:
# Check for any missing data
print("Missing values per column:")
print(combined_df.isnull().sum())

# Save to CSV
output_path = '../data/raw/Vegetable_prices_weekly.csv'
combined_df.to_csv(output_path, index=False)

print(f"File saved to {output_path}")

Missing values per column:
year                              0
week                              0
vegetable                         0
price                           492
Badulla_actual_class            132
Hambantota_actual_class         132
Jaffna_actual_class             132
Kurunegala_actual_class         132
Matale_actual_class             132
Nuwara_Eliya_actual_class       132
Ratnapura_actual_class          132
Badulla_precipitation           132
Hambantota_precipitation        132
Jaffna_precipitation            132
Kurunegala_precipitation        132
Matale_precipitation            132
Nuwara_Eliya_precipitation      132
Ratnapura_precipitation         132
Badulla_prob_drought            132
Hambantota_prob_drought         132
Jaffna_prob_drought             132
Kurunegala_prob_drought         132
Matale_prob_drought             132
Nuwara_Eliya_prob_drought       132
Ratnapura_prob_drought          132
Badulla_prob_flood_risk         132
Hambantota_prob_flood_risk      132
J

In [68]:
# Read back and check
check_df = pd.read_csv(output_path)
print("Loaded saved file shape:", check_df.shape)
print("First 20 rows:")
display(check_df.head(20))

Loaded saved file shape: (5088, 39)
First 20 rows:


,year,week,vegetable,price,Badulla_actual_class,Hambantota_actual_class,Jaffna_actual_class,Kurunegala_actual_class,Matale_actual_class,Nuwara_Eliya_actual_class,...,Matale_prob_flood_risk,Nuwara_Eliya_prob_flood_risk,Ratnapura_prob_flood_risk,Badulla_prob_normal,Hambantota_prob_normal,Jaffna_prob_normal,Kurunegala_prob_normal,Matale_prob_normal,Nuwara_Eliya_prob_normal,Ratnapura_prob_normal
0,2010,W1,Bitter Gourd,52.4,drought,normal,drought,drought,drought,drought,...,0.000000,0.0,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
1,2010,W1,Brinjals,60.6,drought,normal,drought,drought,drought,drought,...,0.000000,0.0,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
2,2010,W1,Cabbage,38.2,drought,normal,drought,drought,drought,drought,...,0.000000,0.0,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
3,2010,W1,Carrot,66.0,drought,normal,drought,drought,drought,drought,...,0.000000,0.0,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
4,2010,W1,Pumpkin,20.4,drought,normal,drought,drought,drought,drought,...,0.000000,0.0,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
5,2010,W1,Tomatoes,51.0,drought,normal,drought,drought,drought,drought,...,0.000000,0.0,0.000000,0.004286,0.994286,0.017143,0.015714,0.015714,0.035714,0.967143
6,2010,W2,Bitter Gourd,58.6,drought,normal,drought,drought,drought,drought,...,0.002857,0.0,0.001429,0.012857,0.911429,0.040000,0.047143,0.050000,0.055714,0.905714
7,2010,W2,Brinjals,51.6,drought,normal,drought,drought,drought,drought,...,0.002857,0.0,0.001429,0.012857,0.911429,0.040000,0.047143,0.050000,0.055714,0.905714
8,2010,W2,Cabbage,41.0,drought,normal,drought,drought,drought,drought,...,0.002857,0.0,0.001429,0.012857,0.911429,0.040000,0.047143,0.050000,0.055714,0.905714
9,2010,W2,Carrot,68.6,drought,normal,drought,drought,drought,drought,...,0.002857,0.0,0.001429,0.012857,0.911429,0.040000,0.047143,0.050000,0.055714,0.905714
